<a href="https://colab.research.google.com/github/hongyan99/llm_learning/blob/main/unsloth_fine_tuning/Llama3_1_%288B%29_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

%pip install "unsloth==2025.2.12" vllm
%pip install --upgrade pillow
# If you are running this notebook on local, you need to install `diffusers` too
%pip install diffusers
# Temporarily install a specific TRL nightly version
%pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

In [3]:
import torch
print(torch.cuda.is_available())

True


### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [3]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Load up `Llama 3.1 8B Instruct`, and set parameters

In [4]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 12.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 54.89%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 12.0 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 0.41 GB. Also swap space = 6 GB.
INFO 02-15 21:58:50 config.py:542] This model supports multiple tasks: {'embed', 'score', 'generate', 'classify', 'reward'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config usin

[W215 21:58:59.043790492 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 02-15 21:59:00 loader.py:1102] Loading weights with BitsAndBytes quantization.  May take a while ...
INFO 02-15 21:59:01 weight_utils.py:252] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.29s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.29s/it]

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.25s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.25s/it]



INFO 02-15 22:00:46 model_runner.py:1115] Loading model weights took 5.5976 GB
INFO 02-15 22:00:46 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-15 22:01:01 worker.py:267] Memory profiling takes 14.36 seconds
INFO 02-15 22:01:01 worker.py:267] the current vLLM instance can use total_gpu_memory (12.00GiB) x gpu_memory_utilization (0.55) = 6.59GiB
INFO 02-15 22:01:01 worker.py:267] model weights take 5.60GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.59GiB; the rest of the memory reserved for KV Cache is 0.36GiB.
INFO 02-15 22:01:01 executor_base.py:110] # CUDA blocks: 186, # CPU blocks: 3072
INFO 02-15 22:01:01 executor_base.py:115] Maximum concurrency for 512 tokens per request: 5.81x
INFO 02-15 22:01:02 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occur

Capturing CUDA graph shapes: 100%|██████████| 19/19 [00:15<00:00,  1.20it/s]

INFO 02-15 22:01:18 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.53 GiB
INFO 02-15 22:01:18 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 31.17 seconds



Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [5]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

Map: 100%|██████████| 7473/7473 [00:00<00:00, 30496.06 examples/s]


<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [ ]:
from trl import GRPOConfig, GRPOTrainer

target_steps = 250   # Increase to new max if resuming training

training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = target_steps,   # Increase to new max if resuming training
    save_steps = 250,   # Interval to save checkpoints
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()
# Comment the above line but nncomment the below to resume training from a checkpoint
# trainer.train(resume_from_checkpoint="outputs/checkpoint-250")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 250
 "-____-"     Number of trainable parameters = 83,886,080


-------------------- Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
Let's break down the problem:

1. The first 10 tickets will be full price ($40 each).
2. The next 2 tickets (12 - 10 = 2) receive a 5% discount.

First, calculate the cost of the first 10 tickets:
10 tickets * $40 = $400

Now, calculate the cost of the 2 tickets with the discount:

The discount on each ticket is 5% of $40 = $2.
The price of each ticket after the discount = $40 - $2 = $38.
2 tickets * $38 = $76

Now, sum the cost of the full price tickets and the discounted tickets:
$400 + $76 = $476

Therefore, Mr. Benson paid $476. 
Extracted:
Let's break down the problem:

1. The first 10 tickets will be full price ($40 each).
2. The next 2 tickets (12 - 10 = 2) receive a 5% discount.

First, calculate the cost of the first 10 tickets:
10 tickets * $40 = $400

Now, calcul

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,-0.000000,0.347167,0.850381,178.333344,0.000000,-0.069500,0.000000,0.000000,0.083333,0.333333
2,0.000000,0.006667,0.016330,189.833344,0.000000,0.006667,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.020833,0.051031,185.166672,0.000249,0.020833,0.000000,0.000000,0.000000,0.000000
4,0.000000,-0.023667,0.057971,197.333344,0.000411,-0.023667,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.021000,0.051439,112.666672,0.000350,0.021000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.388000,0.950402,196.333344,0.000310,-0.028667,0.000000,0.000000,0.083333,0.333333
7,0.000000,0.020667,0.050623,181.333344,0.000390,0.020667,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.102000,0.144798,120.666672,0.000463,0.018667,0.000000,0.000000,0.083333,0.000000
9,0.000000,-0.044833,0.178214,167.000000,0.000466,-0.044833,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.000000,0.000000,199.833344,0.000286,0.000000,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Jane is trying to decide whether to buy a house or a trailer. A house costs $480,000 and a trailer costs $120,000. Each loan will be paid in monthly installments over 20 years. How much more is the monthly payment on the house compared to the trailer? 
Answer:
1500 
Response:
To find the monthly payment difference between the two options, we need to first calculate the monthly payments for both the house and the trailer.

The monthly payment formula is:
\[ M = P \times \frac{r(1 + r)^n}{(1 + r)^n - 1} \]
where M is the monthly payment, P is the principal amount, r is the monthly interest rate (annual rate divided by 12), and n is the number of payments.

Since we don't have the interest rates, let's assume a fixed interest rate of 4% annually for both the house and the trailer.

For the house:
P = $480,000, r = 0.04/12, n = 20 \* 12 = 240 months

\[ M_{house} = 480,000 \times \frac{(0.04/12)(1 + 0.04/12)^{240}}{(1 + 0.04/ 
Extracted:
To find the monthly p

TrainOutput(global_step=250, training_loss=0.0010621354778450112, metrics={'train_runtime': 7971.7233, 'train_samples_per_second': 0.031, 'train_steps_per_second': 0.031, 'total_flos': 0.0, 'train_loss': 0.0010621354778450112})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [8]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:14<00:00, 14.03s/it, est. speed input: 2.78 toks/s, output: 33.79 toks/s]


'Calculating pi is a complex task that requires a deep understanding of mathematics, particularly in the fields of geometry and analysis. There are several ways to calculate pi, including:\n\n1. **Archimedes\' Method**: This method, developed by the ancient Greek mathematician Archimedes, involves inscribing and circumscribing polygons around a circle to estimate the value of pi. By calculating the perimeter of the polygons and the diameter of the circle, Archimedes was able to establish a bound on the value of pi.\n\n2. **Leibniz Formula**: This formula, developed by the German mathematician Gottfried Wilhelm Leibniz, is a series expansion that approximates the value of pi. The formula is:\n\n   π/4 = 1 - 1/3 + 1/5 - 1/7 + 1/9 - ...\n\n3. **Monte Carlo Method**: This method uses random sampling to estimate the value of pi. The idea is to generate random points within a square and count the number of points that fall within a circle inscribed within the square. The ratio of the number 

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [13]:
%pwd

'/mnt/d/dev/llm_learning/unsloth_fine_tuning'

In [9]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [10]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:14<00:00, 14.93s/it, est. speed input: 4.09 toks/s, output: 30.28 toks/s]


'## Calculating Pi\n\nPi (π) is a mathematical constant representing the ratio of a circle\'s circumference to its diameter. It is an irrational number, approximately equal to 3.14159 (but it is a transcendental number, which means it cannot be expressed exactly as a finite decimal or fraction).\n\n### Numerical Methods for Calculating Pi\n\nThere are various methods to calculate pi, ranging from simple geometric constructions to advanced mathematical techniques.\n\n### Geometric Construction Method\n\nOne simple method to estimate pi is by drawing a circle and inscribing a regular polygon (e.g., hexagon or octagon) inside it. The perimeter of the polygon is then divided by its apothem (the distance from the center of the polygon to one of its vertices) to get an approximation of pi.\n\n### Babylonian Method\n\nThis ancient method uses an iterative formula to calculate pi:\n\nπ ≈ (2 * x^2 / (2x - 1))\n\nwhere x_0 is an initial value, typically set to 3.0, and x is updated in each itera

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [20]:
from huggingface_hub import HfApi, create_repo

# Load the token from a file
with open('hf_token.txt', 'r') as file:
	hf_token = file.read().strip()

# Define repository name
repo_name = "meta-llama-3.1-8B-GrpoFusion"
username = "rachmaninoxoxo"  # Replace with your actual Hugging Face username
repo_id = f"{username}/{repo_name}"

# Create repository if it does not exist
api = HfApi()
try:
	api.repo_info(repo_id, token=hf_token)
except:
	create_repo(repo_id, token=hf_token)

# Merge to 16bit
merge_to_16bit = True
if merge_to_16bit: model.save_pretrained_merged("model", tokenizer, save_method="merged_16bit",)
if merge_to_16bit: model.push_to_hub_merged(repo_id, tokenizer, save_method="merged_16bit", token=hf_token)

# Merge to 4bit
merge_to_4bit = False
if merge_to_4bit: model.save_pretrained_merged("model", tokenizer, save_method="merged_4bit",)
if merge_to_4bit: model.push_to_hub_merged(repo_id, tokenizer, save_method="merged_4bit", token=hf_token)

# Just LoRA adapters
lora_adapters_only = False
if lora_adapters_only: model.save_pretrained_merged("model", tokenizer, save_method="lora",)
if lora_adapters_only: model.push_to_hub_merged(repo_id, tokenizer, save_method="lora", token=hf_token)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 32.5 out of 62.76 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [01:27<00:00,  2.74s/it]


Unsloth: Saving tokenizer... Done.
Done.
Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 32.5 out of 62.76 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [01:29<00:00,  2.80s/it]


Unsloth: Saving tokenizer...

No files have been modified since last commit. Skipping to prevent empty commit.


 Done.


No files have been modified since last commit. Skipping to prevent empty commit.


Done.
Saved merged model to https://huggingface.co/rachmaninoxoxo/meta-llama-3.1-8B-GrpoFusion


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Load the token from a file
with open('hf_token.txt', 'r') as file:
	hf_token = file.read().strip()

# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
